In [1]:
import findspark
findspark.init()

import pyspark

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [4]:
from pyspark.sql import SparkSession


spark= SparkSession.builder.appName('dataframe').getOrCreate()

In [6]:
# load the retail dataset:
retail_data=spark.read.csv("online-retail-dataset.csv",inferSchema=True,header=True,timestampFormat="dd/M/yyyy H:mm")

In [7]:
retail_data.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-01-12 08:26:00|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-01-12 08:26:00|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-01-12 08:26:00|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-01-12 08:26:00|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-01-12 08:26:00|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-01-12 08:26:00|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS S

In [8]:
### Question 1 ###
# How many orders did customers perform at which hour?

# a) SQL
retail_data.createOrReplaceTempView("retailTable")


In [10]:
result = spark.sql("""
SELECT hour(InvoiceDate) as InvoiceHour, count(distinct InvoiceNo) as NoInvoices
FROM retailTable
GROUP BY InvoiceHour
ORDER BY InvoiceHour
""")
result.show()

+-----------+----------+
|InvoiceHour|NoInvoices|
+-----------+----------+
|          6|        22|
|          7|        31|
|          8|       624|
|          9|      1824|
|         10|      2961|
|         11|      3165|
|         12|      3962|
|         13|      3369|
|         14|      3137|
|         15|      3069|
|         16|      1952|
|         17|      1205|
|         18|       333|
|         19|       219|
|         20|        28|
+-----------+----------+



In [13]:
# b) Spark
result = retail_data.selectExpr("hour(InvoiceDate) as InvoiceHour", "InvoiceNo").distinct().groupBy('InvoiceHour').agg(f.expr("count(InvoiceNo) as NoInvoices")).orderBy('InvoiceHour')
result.show()

+-----------+----------+
|InvoiceHour|NoInvoices|
+-----------+----------+
|          6|        22|
|          7|        31|
|          8|       624|
|          9|      1824|
|         10|      2961|
|         11|      3165|
|         12|      3962|
|         13|      3369|
|         14|      3137|
|         15|      3069|
|         16|      1952|
|         17|      1205|
|         18|       333|
|         19|       219|
|         20|        28|
+-----------+----------+

